In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import json
import nltk
import re
from nltk.corpus import stopwords
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense, Dropout

In [ ]:
df = pd.read_csv('/content/drive/MyDrive/NLP/finaldataset.csv')

In [ ]:
df.head()

,data,title_x,title_y,title,value,tag
0,লিখার সময় পারলে সত্য লিখার অভ্যাস শিখুন।,-1,-1,2,-1,কিছুটা নেতিবাচক
1,এটা কেন হচ্ছে? সংশ্লিষ্ট সকলের ডিপ্রেশনের ফলে?...,-1,-1,-1,-1,কিছুটা নেতিবাচক
2,আমাদের দেশের স্বাভাবিক অর্থনৈতিক গতিপ্রবাহকে ব...,-1,-2,-2,-5,নিশ্চিত নেতিবাচক
3,চুরি নয় লুটপাট।,-2,-2,-2,-6,নিশ্চিত নেতিবাচক
4,ইসলামী ব্যাংকের বর্তমান অবস্থা দেখে মনে হয় শাস...,0,-1,0,0,নিরপেক্ষ


In [ ]:
df['tag'].value_counts()

,count
tag,
নিশ্চিত নেতিবাচক,3928
কিছুটা নেতিবাচক,3198
নিরপেক্ষ,2951
নিশ্চিত ইতিবাচক,2280
কিছুটা ইতিবাচক,1445


In [ ]:
cnt = df['tag'].nunique()

In [ ]:
cnt

5

In [ ]:
df = df.drop(['title_x','title_y','title','value'],axis=1)

In [ ]:
df.head(30)

,data,tag
0,লিখার সময় পারলে সত্য লিখার অভ্যাস শিখুন।,কিছুটা নেতিবাচক
1,এটা কেন হচ্ছে? সংশ্লিষ্ট সকলের ডিপ্রেশনের ফলে?...,কিছুটা নেতিবাচক
2,আমাদের দেশের স্বাভাবিক অর্থনৈতিক গতিপ্রবাহকে ব...,নিশ্চিত নেতিবাচক
3,চুরি নয় লুটপাট।,নিশ্চিত নেতিবাচক
4,ইসলামী ব্যাংকের বর্তমান অবস্থা দেখে মনে হয় শাস...,নিরপেক্ষ
5,একটা ভাল প্রতিষ্ঠান কিভাবে লুট হয়ে যাচেছ!,নিশ্চিত নেতিবাচক
6,সরকার যাদের এই ব্যাংকে নিয়গ দিয়েছে তারা ব্যাংক...,নিশ্চিত নেতিবাচক
7,পুরোপুরি আওয়ামীকরন হলেই আর এর অসতিতই থাকবেক না!,কিছুটা নেতিবাচক
8,এককথায় ব্যাংকটিতে ডাকাতি হয়েছে এবং হতেই থাকবে।,কিছুটা নেতিবাচক
9,ইসলামি ব্যাংক প্রারম্ভ থেকেই গ্রাহকদের পছন্দের...,কিছুটা নেতিবাচক


In [ ]:
with open('/content/drive/MyDrive/NLP/bangla_stopwords.txt', 'r', encoding='utf-8') as f:
    bn_stopwords = f.read().splitlines()

In [ ]:
print(bn_stopwords)

['আমি', 'আপনি', 'সে', 'তুমি', 'আমরা', 'আপনারা', 'তারা', 'এর', 'এই', 'ওই', 'তার', 'তাদের', 'আমাদের', 'আপনার', 'তোমার', 'আপনারা', 'আমরা', 'যে', 'যত', 'সব', 'কিছু', 'বহু', 'কোনো', 'কোন', 'একটি', 'এটি', 'তিন', 'চার', 'এখানে', 'যেখানে', 'এখানে', 'কিভাবে', 'কীভাবে', 'কেন', 'কেননা', 'যখন', 'যদি', 'তবে', 'কিন্তু', 'তবে', 'আর', 'তিন', 'চার', 'অথবা', 'নাহলে', 'যত', 'যেমন', 'কিন্তু', 'নিশ্চিত', 'সবাই', 'অনেক', 'কিছু', 'কেননা', 'অন্য', 'নতুন', 'পুরানো', 'বেশি', 'কম', 'অন্তত', 'বেশি', 'কম', 'এবং', 'অথবা', 'নির্বাচন', 'এই', 'ওই', 'ফিরে', 'তারপর', 'পরে', 'আগে', 'তখন', 'নতুন', 'পুরানো', 'যে', 'তবে', 'অথবা', 'একটি', 'অথবা', 'অবশ্য', 'এরপর', 'আমরা', 'বিভিন্ন', 'সকল', 'যেখানে', 'এখানে', 'কি', 'মাঝে', 'মধ্যে', 'মধ্যবর্তী', 'যে', 'শুধু', 'উল্লেখযোগ্য', 'অধিক', 'যেমন', 'বিভিন্ন', 'অপর', 'অন্য', 'কিছু', 'অন্যান্য', 'আর', 'যাওয়া', 'আসা', 'কী', 'যখন', 'এটি', 'কারণে', 'তারপর', 'তাদের', 'আমাদের', 'আরও', 'অবশ্যই', 'এবং', 'অথবা', 'বিশেষ', 'কি', 'ব্যাপারে', 'অথবা', 'দ্বারা', 'তারা', 'এক', 'মধ্যে', 'কিছু', 'তবে', '

In [ ]:
len(bn_stopwords)

1351

In [ ]:
unique_bn_stopwords = set(bn_stopwords)

In [ ]:
len(unique_bn_stopwords)

427

In [ ]:
print(unique_bn_stopwords)

{'জানায়', 'ওখানে', 'তারা', 'সঙ্গে', 'ধরা', 'ছাড়া', 'করে', 'আদ্যভাগে', 'ওরা', 'হওয়ায়', 'করায়', 'এতটাই', 'সব', 'একটি', 'করেন', 'এমনি', 'করেছে', 'এই', 'বেশ', 'ওঁদের', 'বলল', 'ক্ষেত্রে', 'কিংবা', 'দিয়েছে', 'শুধু', 'এল', 'এমন', 'আপনারা', 'যার', 'টি', 'করবে', 'ই', 'চার', 'নেওয়া', 'আই', 'বলে', 'পর্যন্ত', 'লক্ষ', 'সবাই', 'কিভাবে', 'তাহাতে', 'ব্যবহার', 'সকল', 'সর্বনিম্ন', 'তাও', 'পেয়ে', 'নানা', 'ওদের', 'অর্থাত', 'করেছেন', 'হচ্ছে', 'সেই', 'এটি', 'হবেন', 'গেল', 'কেউই', 'এবার', 'অবশ্যই', 'ভেতরে', 'আগেই', 'সবার', 'অপর', 'করেছিলেন', 'সুতরাং', 'অন্তত', 'মাঝে', 'তবু', 'তাঁাহারা', 'পারি', 'অন্য', 'হোক', 'প্রতি', 'এটাই', 'এরা', 'যাঁর', 'বিভিন্ন', 'হত', 'ওঁর', 'যখন', 'ওর', 'দুই', 'হলে', 'মধ্যবর্তী', 'অন্যান্য', 'তত', 'আবার', 'কিন্তু', 'মধ্যেই', 'হয়ে', 'নতুন', 'বরং', 'আমাদের', 'থাকায়', 'হইয়া', 'দুটি', 'এঁদের', 'সামনে', 'ঠিক', 'দিলেন', 'নয়', 'কাছ', 'এত', 'করছে', 'কবে', 'গুলি', 'গিয়ে', 'জন', 'দিয়ে', 'অথবা', 'অনেকেই', 'এখন', 'দেখতে', 'যাওয়া', 'নিতে', 'জনকে', 'আমরা', 'করিতে', 'বহু', 'থেকেও', 'করলে

In [ ]:
with open('/content/drive/MyDrive/NLP/emoji_dictionary.json','r',encoding='utf-8') as f:
     emoji_dict = json.load(f)

In [ ]:
print(emoji_dict)

{'😊': 'হাসি', '😢': 'কান্না', '😠': 'রাগ', '😔': 'মন খারাপ', '👍': 'দারুণ', '😎': 'ঠান্ডা', '😭': 'অশ্রু', '😁': 'মুচকি হাসি', '😅': 'হালকা হাসি', '😍': 'ভালবাসা', '😒': 'অসন্তুষ্ট', '😞': 'হতাশা', '😡': 'রাগান্বিত', '😃': 'খুশি', '😉': 'চোখ মারা', '😋': 'স্বাদ আস্বাদন', '😐': 'নির্বিকার', '😤': 'অসন্তুষ্ট', '😴': 'ঘুম', '😜': 'মজার', '😩': 'ক্লান্ত', '😯': 'আশ্চর্য', '😆': 'হাসি', '😷': 'মাস্ক', '🙄': 'গুর্গুরানি', '😳': 'বিস্মিত', '😬': 'চিন্তিত', '😚': 'চুম্বন', '😰': 'উদ্বিগ্ন', '🤗': 'আলিঙ্গন', '🤔': 'চিন্তাশীল', '🤐': 'মুখ বন্ধ', '😇': 'পুণ্যবান'}


In [ ]:
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [ ]:
df = df[(df['tag']=='নিশ্চিত নেতিবাচক')|(df['tag']=='নিশ্চিত ইতিবাচক')]

In [ ]:
df['tag'].value_counts()

,count
tag,
নিশ্চিত নেতিবাচক,3928
নিশ্চিত ইতিবাচক,2280


In [ ]:
import nltk

def remove_stopwords(text):
    words = nltk.word_tokenize(text)
    filtered_words = [word for word in words if word not in bn_stopwords]
    return ' '.join(filtered_words)

def remove_non_bangla_words(text):
    return ' '.join([word for word in text.split() if all('ঀ' <= c <= '৿' for c in word)])

def remove_null_characters(text):
    return text.replace('\x00', '')

def remove_urls(text):
    url_pattern = re.compile(r'https?://\S+|www\.\S+')
    return url_pattern.sub(r'', text)


def replace_emojis(text):
    for emoji, meaning in emoji_dict.items():
        text = text.replace(emoji, meaning)
    return text


def preprocess_text(text):
    text = replace_emojis(text)
    text = remove_stopwords(text)
    text = remove_non_bangla_words(text)
    text = remove_null_characters(text)
    text = remove_urls(text)
    return text

df['processed_text'] = df['data'].apply(preprocess_text)


In [ ]:
df.sample(60)

,data,tag,processed_text
11499,"আমি গিয়ে দেখি স্কুলে প্রবেশের সব গেট বন্ধ, মা...",নিশ্চিত নেতিবাচক,দেখি স্কুলে প্রবেশের গেট বন্ধ গেটের নৌকার মিলে...
9723,ইংল্যান্ডের সংবাদ মাধ্যমকে আরও সতর্ক ও সংযত হয়...,নিশ্চিত নেতিবাচক,ইংল্যান্ডের সংবাদ মাধ্যমকে সতর্ক সংযত হয়ে কথা ...
6116,ইদানিং মনে হচ্ছে আপনি নিজের ঢোলটা যেন একটু ব...,নিশ্চিত নেতিবাচক,ইদানিং ঢোলটা একটু বেশীই বাজাচ্ছেন
12155,খুব মনোযোগ দিয়ে পড়লাম চিঠিখানা। পড়তে কষ্ট হচ্ছ...,নিশ্চিত ইতিবাচক,মনোযোগ দিয়ে পড়লাম পড়তে কষ্ট কখন চোখটা ভিজে টের...
10661,অনেকে ৭ নম্বরে লিটনকে বিবেচনা করছে কিন্তু সেখা...,নিশ্চিত ইতিবাচক,৭ নম্বরে লিটনকে বিবেচনা সাইফুদ্দিন স্বপ্ন দেখছ...
10947,যার আশায় বসে ছিল সেই রোনালদো দেখল হলুদ কার্ড...,নিশ্চিত নেতিবাচক,আশায় রোনালদো দেখল হলুদ কার্ড রামোস দেখল লাল
4837,বোঝা গেল সুযোগ পেলেই একজন রাজনীতিবিদ অন্য একজন...,নিশ্চিত নেতিবাচক,বোঝা সুযোগ পেলেই একজন রাজনীতিবিদ একজন রাজনীতিব...
4963,আপনাদের বিএনপির অভিষন্ধী কখনো ভাল ছিলনা তা আর ...,নিশ্চিত নেতিবাচক,আপনাদের বিএনপির অভিষন্ধী কখনো ভাল ছিলনা গোপন ক...
2845,পাকিস্তানে নির্বাচনকালীন তত্ববধ্যায়ক সরকার আছে...,নিশ্চিত নেতিবাচক,পাকিস্তানে নির্বাচনকালীন তত্ববধ্যায়ক সরকার সেন...
13509,গাছেরও যে প্রাণ আছে সেটা বাঙালি বিজ্ঞানী জগদীশ...,নিশ্চিত ইতিবাচক,গাছেরও প্রাণ বাঙালি বিজ্ঞানী জগদীশ বসু আবিষ্কা...


In [ ]:
df['tag'].value_counts()

,count
tag,
নিশ্চিত নেতিবাচক,3928
নিশ্চিত ইতিবাচক,2280


In [ ]:
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split

# Encoding the labels
encoder = LabelEncoder()
df['label'] = encoder.fit_transform(df['tag'])

# Tokenizing and padding the sequences
tokenizer = Tokenizer()
tokenizer.fit_on_texts(df['processed_text'])
sequences = tokenizer.texts_to_sequences(df['processed_text'])
max_length = max([len(x) for x in sequences])
X = pad_sequences(sequences, maxlen=max_length, padding='post')
y = df['label']

# Splitting the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, stratify=y, random_state=42)


In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, Conv1D, GlobalMaxPooling1D, Dense, Dropout

# Model parameters
vocab_size = len(tokenizer.word_index) + 1
embedding_dim = 64
max_length = X.shape[1]

# Building the CNN model
model = Sequential([
    Embedding(input_dim=vocab_size, output_dim=embedding_dim, input_length=max_length),
    Conv1D(128, 5, activation='relu'),
    GlobalMaxPooling1D(),
    Dropout(0.5),
    Dense(64, activation='relu'),
    Dense(2, activation='softmax')  # 3 classes: happy, sad, angry
])

model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Training the model
history = model.fit(X_train, y_train, epochs=15, validation_data=(X_test, y_test), batch_size=32)


Epoch 1/15


/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


156/156 ━━━━━━━━━━━━━━━━━━━━ 7s 31ms/step - accuracy: 0.6092 - loss: 0.6633 - val_accuracy: 0.6329 - val_loss: 0.6401
Epoch 2/15
156/156 ━━━━━━━━━━━━━━━━━━━━ 4s 24ms/step - accuracy: 0.7154 - loss: 0.5346 - val_accuracy: 0.7045 - val_loss: 0.5731
Epoch 3/15
156/156 ━━━━━━━━━━━━━━━━━━━━ 4s 23ms/step - accuracy: 0.9473 - loss: 0.1713 - val_accuracy: 0.7037 - val_loss: 0.7811
Epoch 4/15
156/156 ━━━━━━━━━━━━━━━━━━━━ 6s 38ms/step - accuracy: 0.9837 - loss: 0.0543 - val_accuracy: 0.6812 - val_loss: 0.9998
Epoch 5/15
156/156 ━━━━━━━━━━━━━━━━━━━━ 4s 24ms/step - accuracy: 0.9942 - loss: 0.0219 - val_accuracy: 0.6948 - val_loss: 1.1470
Epoch 6/15
156/156 ━━━━━━━━━━━━━━━━━━━━ 4s 25ms/step - accuracy: 0.9955 - loss: 0.0257 - val_accuracy: 0.6924 - val_loss: 1.1791
Epoch 7/15
156/156 ━━━━━━━━━━━━━━━━━━━━ 5s 30ms/step - accuracy: 0.9939 - loss: 0.0228 - val_accuracy: 0.6924 - val_loss: 1.2272
Epoch 8/15
156/156 ━━━━━━━━━━━━━━━━━━━━ 5s 29ms/step - accuracy: 0.9954 - loss: 0.0161 - val_accuracy: 0.697

In [ ]:
def predict_sentiment(text):
    # Preprocess the text
    text_seq = tokenizer.texts_to_sequences([text])
    text_pad = pad_sequences(text_seq, maxlen=max_length, padding='post')

    # Predict the sentiment
    prediction = model.predict(text_pad)
    predicted_label = encoder.inverse_transform([prediction.argmax(axis=1)[0]])[0]

    return predicted_label

# Example usage
new_text = "ভালো সিদ্ধান্ত"
predicted_label = predict_sentiment(new_text)
print(f"The predicted sentiment for the text is: {predicted_label}")


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 80ms/step
The predicted sentiment for the text is: নিশ্চিত ইতিবাচক
